# Image Segmentation Task 1
#### Welcome to the first task of Image Segmentation. Image segmentation is the process of partitioning the image into a set of pixels representing an object. In this task, you will be introduced to the problem of image segmentation and programming pipeline involved in image segmentation.

For the purpose of this task we will be using PASCAL VOC datset. The dataset contains a total of 2913 images with segmentation annotations. Code in the cell below will download the code and extract the dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd "/content/drive/MyDrive/NN_Project"

In [ ]:
pwd

In [ ]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
!tar -xvf VOCtrainval_11-May-2012.tar

In [ ]:
!sudo pip install --upgrade scipy==1.1.0

### 1.1 Loading the dataset

In [4]:
import os
from os.path import join as pjoin
import collections
import json
import torch
import imageio
import numpy as np
import scipy.misc as m
import scipy.io as io
import matplotlib.pyplot as plt
import glob

from PIL import Image
from tqdm import tqdm
from torch.utils import data
from torchvision import transforms


class pascalVOCDataset(data.Dataset):
    """Data loader for the Pascal VOC semantic segmentation dataset.

    Annotations from both the original VOC data (which consist of RGB images
    in which colours map to specific classes) and the SBD (Berkely) dataset
    (where annotations are stored as .mat files) are converted into a common
    `label_mask` format.  Under this format, each mask is an (M,N) array of
    integer values from 0 to 21, where 0 represents the background class.

    The label masks are stored in a new folder, called `pre_encoded`, which
    is added as a subdirectory of the `SegmentationClass` folder in the
    original Pascal VOC data layout.

    A total of five data splits are provided for working with the VOC data:
        train: The original VOC 2012 training data - 1464 images
        val: The original VOC 2012 validation data - 1449 images
        trainval: The combination of `train` and `val` - 2913 images
        train_aug: The unique images present in both the train split and
                   training images from SBD: - 8829 images (the unique members
                   of the result of combining lists of length 1464 and 8498)
        train_aug_val: The original VOC 2012 validation data minus the images
                   present in `train_aug` (This is done with the same logic as
                   the validation set used in FCN PAMI paper, but with VOC 2012
                   rather than VOC 2011) - 904 images
    """

    def __init__(
        self,
        root,
        sbd_path=None,
        split="train_aug",
        is_transform=False,
        img_size=512,
        augmentations=None,
        img_norm=True,
        test_mode=False,
    ):
        self.root = root
        self.sbd_path = sbd_path
        self.split = split
        self.is_transform = is_transform
        self.augmentations = augmentations
        self.img_norm = img_norm
        self.test_mode = test_mode
        self.n_classes = 21
        self.mean = np.array([104.00699, 116.66877, 122.67892])
        self.files = collections.defaultdict(list)
        self.img_size = img_size if isinstance(img_size, tuple) else (img_size, img_size)

        if not self.test_mode:
            for split in ["train", "val", "trainval"]:
                path = pjoin(self.root, "ImageSets/Segmentation", split + ".txt")
                file_list = tuple(open(path, "r"))
                file_list = [id_.rstrip() for id_ in file_list]
                self.files[split] = file_list
            self.setup_annotations()

        self.tf = transforms.Compose(
            [
                # add more trasnformations as you see fit
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )

    def __len__(self):
        return len(self.files[self.split])

    def __getitem__(self, index):
        im_name = self.files[self.split][index]
        im_path = pjoin(self.root, "JPEGImages", im_name + ".jpg")
        lbl_path = pjoin(self.root, "SegmentationClass/pre_encoded", im_name + ".png")
        im = Image.open(im_path)
        lbl = Image.open(lbl_path)
        if self.augmentations is not None:
            im, lbl = self.augmentations(im, lbl)
        if self.is_transform:
            im, lbl = self.transform(im, lbl)
        return im, torch.clamp(lbl, max=20)

    def transform(self, img, lbl):
        if self.img_size == ("same", "same"):
            pass
        else:
            img = img.resize((self.img_size[0], self.img_size[1]))  # uint8 with RGB mode
            lbl = lbl.resize((self.img_size[0], self.img_size[1]))
        img = self.tf(img)
        lbl = torch.from_numpy(np.array(lbl)).long()
        lbl[lbl == 255] = 0
        return img, lbl

    def get_pascal_labels(self):
        """Load the mapping that associates pascal classes with label colors

        Returns:
            np.ndarray with dimensions (21, 3)
        """
        return np.asarray(
            [
                [0, 0, 0],
                [128, 0, 0],
                [0, 128, 0],
                [128, 128, 0],
                [0, 0, 128],
                [128, 0, 128],
                [0, 128, 128],
                [128, 128, 128],
                [64, 0, 0],
                [192, 0, 0],
                [64, 128, 0],
                [192, 128, 0],
                [64, 0, 128],
                [192, 0, 128],
                [64, 128, 128],
                [192, 128, 128],
                [0, 64, 0],
                [128, 64, 0],
                [0, 192, 0],
                [128, 192, 0],
                [0, 64, 128],
            ]
        )

    def encode_segmap(self, mask):
        """Encode segmentation label images as pascal classes

        Args:
            mask (np.ndarray): raw segmentation label image of dimension
              (M, N, 3), in which the Pascal classes are encoded as colours.

        Returns:
            (np.ndarray): class map with dimensions (M,N), where the value at
            a given location is the integer denoting the class index.
        """
        mask = mask.astype(int)
        label_mask = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.int16)
        for ii, label in enumerate(self.get_pascal_labels()):
            label_mask[np.where(np.all(mask == label, axis=-1))[:2]] = ii
        label_mask = label_mask.astype(int)
        # print(np.unique(label_mask))
        return label_mask

    def decode_segmap(self, label_mask, plot=False):
        """Decode segmentation class labels into a color image

        Args:
            label_mask (np.ndarray): an (M,N) array of integer values denoting
              the class label at each spatial location.
            plot (bool, optional): whether to show the resulting color image
              in a figure.

        Returns:
            (np.ndarray, optional): the resulting decoded color image.
        """
        label_colours = self.get_pascal_labels()
        r = label_mask.copy()
        g = label_mask.copy()
        b = label_mask.copy()
        for ll in range(0, self.n_classes):
            r[label_mask == ll] = label_colours[ll, 0]
            g[label_mask == ll] = label_colours[ll, 1]
            b[label_mask == ll] = label_colours[ll, 2]
        rgb = np.zeros((label_mask.shape[0], label_mask.shape[1], 3))
        rgb[:, :, 0] = r / 255.0
        rgb[:, :, 1] = g / 255.0
        rgb[:, :, 2] = b / 255.0
        if plot:
            plt.imshow(rgb)
            plt.show()
        else:
            return rgb

    def setup_annotations(self):
        """Sets up Berkley annotations by adding image indices to the
        `train_aug` split and pre-encode all segmentation labels into the
        common label_mask format (if this has not already been done). This
        function also defines the `train_aug` and `train_aug_val` data splits
        according to the description in the class docstring
        """
        sbd_path = self.sbd_path
        target_path = pjoin(self.root, "SegmentationClass/pre_encoded")
        if not os.path.exists(target_path):
            os.makedirs(target_path)
        train_aug = self.files["train"]

        # keep unique elements (stable)
        train_aug = [train_aug[i] for i in sorted(np.unique(train_aug, return_index=True)[1])]
        self.files["train_aug"] = train_aug
        set_diff = set(self.files["val"]) - set(train_aug)  # remove overlap
        self.files["train_aug_val"] = list(set_diff)

        pre_encoded = glob.glob(pjoin(target_path, "*.png"))
        expected = np.unique(self.files["train_aug"] + self.files["val"]).size

        if len(pre_encoded) != expected:
            print("Pre-encoding segmentation masks...")

            for ii in tqdm(self.files["trainval"]):
                fname = ii + ".png"
                lbl_path = pjoin(self.root, "SegmentationClass", fname)
                lbl = self.encode_segmap(m.imread(lbl_path))
                lbl = m.toimage(lbl, high=lbl.max(), low=lbl.min())
                m.imsave(pjoin(target_path, fname), lbl)

        assert expected == 2913, "unexpected dataset sizes"

### 1.2 Define the model architecture(2.0 point)
In this section you have the freedom to decide your own model. Keep in mind though, to perform image segmentation, you need to implement an architecture that does pixel level classification i.e. for each pixel in the image you need to predict the probability of it belonging to one of the 21 categories.

In [5]:
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F

#Taking the Vgg_16 pre-trained model and adding the last layer to get the same output size as input.

class Vgg_16(nn.Module):
  
    def __init__(self, num_classes):
        super(Vgg_16, self).__init__()
        
        #Storing the features of the vgg16 pretrained model
        self.fea = models.vgg16(pretrained=True).features
         
        #Adding the last layer to make the output size same as input
        self.layer = nn.Sequential(
            nn.Conv2d(512, 4096, 3),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Conv2d(4096, 4096, 3),
            nn.ReLU(inplace=True),
            nn.Dropout(),)
        
        #output layer
        self.score = nn.Conv2d(4096, num_classes, 1)
    
    def forward(self, x):
        #define the forward pass
        A1 = self.fea(x)
        A2 = self.layer(A1)
        A3 = self.score(A2)

        return F.upsample_bilinear(A3, x.size()[2:])




In [ ]:
# Creating an instance of the model defined above. 
# You can modify it incase you need to pass paratemers to the constructor.
model = Vgg_16(21).cuda()

### 1.3 Hyperparameters(0.5 points)
Define all the hyperparameters(not restricted to the three given below) that you find useful here.

In [6]:
local_path = 'VOC2012/' # modify it according to your device
bs = 10
epochs =30

### 1.4 Dataset and Dataloader(0.5 points)
Create the dataset using pascalVOCDataset class defined above. Use local_path defined in the cell above as root. 

In [ ]:
# dataset variable
dst = pascalVOCDataset('VOC2012/', is_transform= True)

# dataloader variable
trainloader = torch.utils.data.DataLoader(dst, bs,shuffle=True)

### 1.5 Loss fuction and Optimizer(1.0 point)
Define below with the loss function you think would be most suitable for segmentation task. You are free to choose any optimizer to train the network.

In [ ]:
model= Vgg_16(21)
model.load_state_dict(torch.load('epoch-14.pt'))
model.eval()
model.cuda()

In [12]:
import torch.optim as optim
# loss function
loss_f = nn.CrossEntropyLoss()

# optimizer variable
opt = optim.Adam(model.parameters(), lr=0.000001, weight_decay=0.00005)

### 1.6 Training the model(3.0 points)
Your task here is to complete the code below to perform a training loop and save the model weights after each epoch of training.

In [ ]:
model_dir= ""
total_loss=0
ctr=0
epoch=5
for e in range(epochs):
  for i, d in enumerate(trainloader):
    # your code goes here
    img,lab= d
    img= img.cuda()
    lab= lab.cuda()
    pred_out= model.forward(img)
    loss= loss_f(pred_out,lab)
    loss.backward()
    opt.step()
    total_loss+= loss.data
    ctr= ctr+1

  print("epoch: {}, loss: {}".format(e, (total_loss/ctr)))
  if e%10==0:
    torch.save(model.state_dict(), os.path.join(model_dir, 'epochs-{}.pt'.format(e)))





### 1.7 Evaluate your model(1.5 points)
In this section you have to implement the evaluation metrics for your model. Calculate the values of F1-score, dice coefficient and AUC-ROC score on the data you used for training. You can use external packages like scikit-learn to compute above metrics.

In [14]:
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import jaccard_score


def evaluate_f1(ground_truth, predictions):
  f1= f1_score(ground_truth.cuda().detach().cpu().clone().numpy().reshape(-1), predictions.cuda().detach().cpu().clone().numpy().argmax(1).reshape(-1), average='micro')

  return f1

def evaluate_auc(ground_truth, predictions):
  ground_truth = torch.flatten(ground_truth, start_dim = 0, end_dim = 2)
  predictions = torch.flatten(predictions, start_dim = 0, end_dim = 2)
  roc_auc= roc_auc_score(ground_truth.cuda().detach().cpu().clone().numpy(), nn.Softmax(predictions.cuda().detach().cpu().clone().numpy().argmax(1)))
  return roc_auc

def evaluate_jacc(ground_truth, predictions):
  jacc = jaccard_score(ground_truth.cuda().detach().cpu().clone().numpy().reshape(-1), predictions.cuda().detach().cpu().clone().numpy().argmax(1).reshape(-1), average='micro')
  return jacc

In [ ]:
f= evaluate_f1(lab, pred_out)
j= evaluate_jacc(lab, pred_out)
print("F1_score:{}, Jaccard_score:{}".format(f,j))

### 1.8 Plot the evaluation metrics against epochs(1.0)
In section 1.6 we saved the weights of the model after each epoch. In this section, you have to calculate the evaluation metrics after each epoch of training by loading the weights for each epoch. Once you have calculated the evaluation metrics for each epoch, plot them against the epochs.

In [ ]:
total_f1=0
total_ja=0
total_loss=0
ctr=0
f1=[]
jac=[]
epoch=10
for e in range(epoch):
  for i, d in enumerate(trainloader):
    # your code goes here
    img,lab= d
    img= img.cuda()
    lab= lab.cuda()
    pred= model(img)
    loss= loss_f(pred,lab)
    f= evaluate_f1(lab, pred)
    total_f1+= f
    j= evaluate_jacc(lab, pred)
    total_ja+= j
    total_loss+= loss.data
    ctr=ctr+1
  f1.append(total_f1/ctr)
  jac.append(total_ja/ctr)
  print("epoch: {}, loss: {}".format(e, (total_loss/ctr)))







In [ ]:
titles = ['F1-score','Dice-coefficient']
fig,ax= plt.subplots(nrows=1, ncols=2, figsize=(14,7))
metric= [f1, jac]
ep=[1,2,3,4,5,6,7,8,9,10]
for i,j in enumerate(ax.flatten()):
  plt.sca(j)
  plt.title(titles[i])
  plt.xlabel('Epoch')
  plt.ylabel('Score')
  plt.plot(ep,metric[i])

### 1.9 Visualize results(0.5 points)
For any 10 images in the dataset, show the images along the with their segmentation mask.

In [ ]:
val= iter(trainloader)
for _ in range(10):
  fig= plt.figure(figsize=(10,10))
  image,mask= next(val)
  image= image.cuda()
  mask= mask.cuda().detach().cpu().numpy()
  preds= model(image)
  preds= preds.detach().cpu().numpy()
  image= image.cpu()
  fig1= fig.add_subplot(131)
  plt.imshow(image[1].transpose(0,2).transpose(0,1).numpy())
  fig1.title.set_text("Image")
  fig1.axis("off")
  fig2= fig.add_subplot(132)
  plt.imshow(mask[1])
  fig2.title.set_text("Ground_Truth")
  fig2.axis("off")
  fig3= fig.add_subplot(133)
  plt.imshow(preds.argmax(1)[1])
  fig3.title.set_text("Prediction")
  fig3.axis("off")
  plt.show()
